## Estadisticas
En este ejemplo se cargara un archivo CSV de metadatos de los jugadores del mundial FIFA 2014 mediante Pandas y se procede a calcular algunas estadisticas especificas de jugadores de cada pais y en base a los goles marcados. 

   ### Ejecucion

In [1]:
from metaflow import FlowSpec, step, IncludeFile

Se procede a importar los modulos a utilizar desde <b>Metaflow</b>

In [2]:
def script_path(filename):
    import os

    current_folder = globals()['_dh'][0]
    filepath = os.path.join(current_folder,'mundial.csv')
    return os.path.join(filepath, filename)

Se implementa la funcion `os.path.join`, conveniente para obtener la ruta del directorio de una manera mas facil. Esto permite utilizar el tutorial desde cualquier directorio 

In [3]:
class PlayersStatFlow(FlowSpec):

    player_data = IncludeFile("player_file",
                               help="El archivo de los jugadores del mundial",
                               default=script_path("mundial.csv"))

De manera similar a los <b>Parametros</b>, se puede definir un archivo de datos para incluirlo como entrada. Metaflow versiona el archivo y lo hara accesible a todos los pasos directamente a traves del objeto `self`

In [6]:
    @step
    def start(self):
        
        import pandas
        from io import StringIO

        # Cargar la data mediante pandas
        self.dataframe = pandas.read_csv(StringIO(self.player_data), sep='|') # StringIo permite leer cadenas como archivos

        # La columna 'genres' tiene una lista de generos para cada pelicula
        self.Country = {countries for Country \
                        in self.dataframe['Country'] \
                        for countries in Country.split('|')} # Se crea un nuevo diccionario con los paises del csv evitando duplicados

        self.Country = list(self.Country) #Convierte el diccionario en lista

        # Se quiere calcular algunas estadiscitas para cada genero.
        # El argumento 'foreach' permite hacer los calculos en paralelo
        self.next(self.compute_statistics, foreach='Country')


En el step <b>start</b> se comienza por analizar el CSV y a este extraer la informacion necesaria y almacenarla en un diccionario de Python. La lectura del CSV se realiza mediante la herramienta Pandas

En el metodo `self.next` se ejecutan pasos paralelos a travez de ramas. La palabra clave <b>foreach</b>

In [7]:
    @step
    def compute_statistics(self):
        
        self.country = self.input

        print("Computing statics for %s" % self.country)

        # Seleccionar todos los jugadores que pertenezcan al pais especificado.
        # Crear un marco de datos con estos jugadores, y solo con las columnas necesarias
        selector = self.dataframe['Country'].apply(lambda row: self.country in row)
        self.dataframe = self.dataframe[selector]

        self.dataframe = self.dataframe[['Last Name','First Name','Country','Goals']]

        # Obtener estadisticas sobre el numero de goles marcados
        points = [.25, .5, .75]
        self.quartiles = self.dataframe['Goals'].quantile(points).values

        # Unir a los resultados de otros paises
        self.next(self.join)

El step <b>compute_statics</b> se ejecuta de manera paralela en funcion de <b>foreach</b>. Obtiene los jugadores que sean del pais especificado.
<br>
Luego se obtienen las estadisticas de los goles marcados por cada jugador, que se encuentran en el dataframe

En el metodo `self.next` se unen los resultados de los otros jugadores despues de que la ejecucion en paralelo del <b>compute_statics</b> termine


In [8]:
    @step
    def join(self, inputs):
        self.country_stats = {inp.country.lower(): \
                                {'quartiles': inp.quartiles,
                                 'dataframe': inp.dataframe} \
                                 for inp in inputs}
        self.next(self.end)

El step <b>join</b> une los resultados de las ramas paralelas combinandolos en un diccionario. Estos datos seran utilizados en el siguiente ejemplo como informacion base para generar una lista mejorada de peliculas recomendadas.

Al ultimo apunta al paso final del programa

In [9]:
    @step
    def end(self):
        """
        End the flow.

        """
        pass

En el step <b>end</b> en general no realiza ninguna accion, pues el presente codigo es previo a la ejecucion del siguiente ejemplo, dejando listos los datos a utilizarse

`if __name__ == '__main__'`:<br>
    `    PlayersStatsFlow()`

Sentencia para empezar la ejecucion del programa

### Si todos los pasos se ejecutaron correctamente, tendriamos una salida similar a la siguiente:

<img src="jugadores2.png">

En la imagen anterior se puede visualizar la correcta ejecucion del programa, como tambien el uso de los cuatro nucleos del computador empleado, como evidencia del trabajo en paralelo de Metaflow



[Descargar CSV](https://drive.google.com/uc?id=12cTxo-XHeipnLuryAzjtrYIuwzJaulbn&export=download&authuser=0)
<br/>
[Descargar archivo .py](https://drive.google.com/uc?id=18J6PKEZRh8L7jrZDFcah-yC5YmjxnC-f&export=download&authuser=0)




